In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import re
import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import scipy as sp

In [2]:
df = pd.read_excel("parcedPatents1217.xlsx",header=0) # engineは環境によってエラー出たり出なかったりだから、必要に応じて変える


In [3]:
df.head(3)
# df['datetime'] = [dt.strptime(i.split('-')[0],  '%Y.%m.%d') for i in df['公開・公表日']]
# df['datetime'] = df['datetime'].astype(str)

,Unnamed: 0,公開・公表番号,発明の名称,公開・公表日,出願日,原出願日,出願人・権利者名,出願番号,登録番号,筆頭発明者,...,ステイタス,ST,早期請求出願有無,要約,全請求項,詳細な説明,company,datetime,sentence,terms
0,0,登実第3082540号,Ｚ形仕切付段ボール箱,2001.12.14,2001-06-08,NaT,佐藤工業株式会社,実願2001-003729,登実第3082540号,佐藤 浩康,...,存続期間満了による抹消,×,無,【要約】【課題】 強度が劣る原紙を用いたとしても箱の強度が維持される外装用段ボール箱を得んと...,【実用新案登録請求の範囲】【請求項１】 広幅と狭幅の各一対の側板を交互に横方向に連設し、それ...,【考案の詳細な説明】 【０００１】 【考案の属する技術分野】 本考案は、袋菓子または詰...,{'佐藤工業株式会社'},2001-12-14,【考案の詳細な説明】 【０００１】 【考案の属する技術分野】 本考案は、袋菓子または詰...,"{'連設': 8, '仕切り壁': 3, '箱本体': 3, '上下縁': 2, '加圧':..."
1,1,登実第3081641号,吊りフック,2001.11.16,2001-05-10,NaT,"寄神建設株式会社,東亜建設工業株式会社,社団法人日本海上起重技術協会",実願2001-002834,登実第3081641号,石塚 浩次,...,存続期間満了による抹消,×,無,【要約】【課題】 重量物を吊り下げて設置を行う際の、重量物の開放を遠隔操作で容易かつ安全に行...,【実用新案登録請求の範囲】【請求項１】 一端がワイヤで吊り下げ支持される少なくとも一枚のフッ...,【考案の詳細な説明】【０００１】【考案の属する技術分野】 本考案は、クレーン等によりコンクリ...,"{'社団法人日本海上起重技術協会', '東亜建設工業株式会社', '寄神建設株式会社'}",2001-11-16,【考案の詳細な説明】【０００１】【考案の属する技術分野】 本考案は、クレーン等によりコンクリ...,"{'玉掛けワイヤ': 12, 'メッセンジャーワイヤ': 10, '下げワイヤ': 6, '..."
2,2,特開2001-311113,橋脚柱の構築工法,2001.11.09,2001-04-19,1995-06-30,株式会社大林組,特願2001-120638,NaN,大内 一,...,拒絶査定（１年経過）,×,無,【要約】【目的】 作業足場や型枠の組立，解体が不要になる橋脚柱の構築工法の提供。【構成】 構...,【特許請求の範囲】【請求項１】 フーチング部と橋梁との間に上下端を一体化させて構築される橋脚...,【発明の詳細な説明】【０００１】【産業上の利用分野】この発明は、橋脚柱の構築工法および橋脚柱...,{'株式会社大林組'},2001-11-09,【発明の詳細な説明】【０００１】【産業上の利用分野】この発明は、橋脚柱の構築工法および橋脚柱...,"{'構築工法': 16, '作業足場': 6, '鋼管設置工程': 5, 'コンクリート打設..."


In [4]:
df.columns

Index(['Unnamed: 0', '公開・公表番号', '発明の名称', '公開・公表日', '出願日', '原出願日', '出願人・権利者名',
       '出願番号', '登録番号', '筆頭発明者', '発明者名', 'IPC', '筆頭ＩＰＣ', 'Ｆターム', 'FI', 'ステイタス',
       'ST', '早期請求出願有無', '要約', '全請求項', '詳細な説明', 'company', 'datetime',
       'sentence', 'terms'],
      dtype='object')

In [5]:
df = df.drop(columns=['Unnamed: 0','早期請求出願有無', '要約', '全請求項', '詳細な説明', 'ST', 'ステイタス', '原出願日', '出願日', '出願日', '登録番号', '公開・公表番号', 'sentence', '出願人・権利者名', '公開・公表日'])

df = df.rename(columns={'発明の名称': 'patent_name', '筆頭ＩＰＣ': 'headIPC', '出願番号': 'id', 'Ｆターム': 'f_term', 'company': 'createdBy', 'datetime':'createdAt', 'terms':'term', '発明者名':'inventor', '筆頭発明者':'head_inventor', })

In [7]:
ret = df.to_dict(orient='index')
df['createdAt'] = df['createdAt'].astype(str)
print (df.dtypes)

# result = {'key':ret}
# import json
# with open('../data/patent1217.json', 'w') as f:
#     result = json.dump(ret, f, indent=2, ensure_ascii=False)

patent_name      object
id               object
head_inventor    object
inventor         object
IPC              object
headIPC          object
f_term           object
FI               object
createdBy        object
createdAt        object
term             object
dtype: object


In [8]:
import json
with open('../data/patent1217.json', 'w') as f:
    result = json.dump(ret, f, indent=2, ensure_ascii=False)

In [18]:
df['詳細な説明'] = df['詳細な説明'].fillna(' ') #nanを文字列に
df['全請求項'] = df['全請求項'].fillna(' ')
df["company"] = [set(i.split(",")) for i in df['出願人・権利者名']]    #出願名をスプリットしてset型に
df['datetime'] = [dt.strptime(i.split(',')[0],  '%Y.%m.%d') for i in df['公開・公表日']]  #公開・公表日を datetime64[ns] 型（ Timestamp 型）に 、複数あるのは先頭のだけに
# df['sentence'] = df['発明の名称'] +df['要約'] + df['全請求項'] .astype("str") + df['詳細な説明']  #なぜか .astype("str") いれないとエラー
# df['sentence'] = df['発明の名称'] +df['要約'] + df['詳細な説明']  
# df['sentence'] = df['発明の名称'] +df['要約']        #sentenceの対象にする項目も必要に応じて変える
df['sentence'] = df['詳細な説明']        #sentenceの対象にする項目も必要に応じて変える

df = df.drop(columns=['早期請求出願有無', '要約', '全請求項', '詳細な説明', 'ST', 'ステイタス', '原出願日', '出願日', '公開・公表日', '出願日', '登録番号', '公開・公表番号', '出願人・権利者名'])

df = df.rename(columns={'発明の名称': 'patent_name', '筆頭ＩＰＣ': 'headIPC', '出願番号': 'id', 'Ｆターム': 'f_term', 'company': 'createdBy', 'datetime':'createdAt', 'terms':'term', '発明者名':'inventor', '筆頭発明者':'head_inventor', })

In [2]:
c_matrix = sp.sparse.load_npz('c_matrix_allterm.npz')
terms_df = pd.read_csv('terms_df.csv')


In [3]:
terms_df.head()

,Unnamed: 0,0
0,0,°、緯度
1,1,°エルボ
2,2,°カメラ
3,3,°ピッチ
4,4,°フック


In [4]:
c_df = pd.DataFrame(terms_df.values, columns=['',"terms"] )
c_df["document_frequency"] =  np.array(c_matrix.astype(bool).sum(axis=0).reshape(-1,1))
c_df["max_term_frequency"]  = c_matrix.T.max(axis=1).toarray()
c_df["sumof_term_frequency"] = np.array(c_matrix.T.sum(axis=1))
c_df.sort_values('document_frequency', ascending=False) # 場合によってはコメントアウト

,,terms,document_frequency,max_term_frequency,sumof_term_frequency
41334,41334,断面形状,4776,53,12964
29925,29925,外径,3984,131,13722
16619,16619,作用効果,3923,22,7890
25506,25506,回動,3316,123,18753
36687,36687,所定間隔,3282,59,7635
...,...,...,...,...,...
32688,32688,岩石質材料,5,2,6
32685,32685,岩石繊維,5,3,7
32684,32684,岩石用ハンマー,5,1,5
32679,32679,岩盤，,5,1,5


In [5]:
re_obj = r'[↑↓°±℃≫≪≦∞【】！-／：-＠［-｀｛-～、-〜”’%]'
c_df = c_df.replace(re_obj, '', regex=True)

In [6]:
c_df.head()

,,terms,document_frequency,max_term_frequency,sumof_term_frequency
0,0,緯度,6,2,7
1,1,エルボ,9,10,24
2,2,カメラ,7,7,16
3,3,ピッチ,21,4,35
4,4,フック,12,3,19


In [7]:
c_df.to_csv("allterms.csv" )

In [8]:
re_obj = r'[↑↓°±℃≫≪≦∞【】！-／：-＠［-｀｛-～、-〜”’%]'
terms_df = terms_df.replace(re_obj, '', regex=True)

In [9]:
terms_df.head()

,Unnamed: 0,0
0,0,緯度
1,1,エルボ
2,2,カメラ
3,3,ピッチ
4,4,フック


In [10]:
terms_list = terms_df['0'].values.tolist()

In [11]:
terms_list

['緯度',
 'エルボ',
 'カメラ',
 'ピッチ',
 'フック',
 '付近',
 '位相',
 '傾斜',
 '刻み',
 '反転',
 '回動',
 '回転',
 '変更',
 '対称',
 '屈曲',
 '旋回',
 '移動',
 '程度傾斜',
 '転回',
 '間隔',
 '離間',
 '傾斜',
 '仰角',
 '湿度',
 '秒',
 'ｃｍ',
 'ｍｍ',
 '××ｃｍ供試体',
 'メチレンビスメチル',
 'コイル出口水温',
 'コイル出口空気温度',
 '冷却水温度',
 '分間遠心分離',
 '圧力',
 '湿度',
 '相対湿度',
 'ｒｐｍ',
 '上昇',
 '付近',
 '低下',
 '冷水',
 '刻み',
 '加熱',
 '加熱養生',
 '変化',
 '封かん養生',
 '標準養生',
 '水中',
 '水中養生',
 '程度上昇',
 '程度低下',
 '近傍',
 '近辺',
 '間隔',
 '養生',
 'ｍｉｎ',
 'ｒａｄ',
 'ｃｓｉｏ',
 '個',
 'こと',
 '√倍',
 '制御',
 'ｌｂｃ',
 '\u3000\u3000内孔',
 '\u3000\u3000開口',
 '湿潤比重',
 '断面',
 '断面',
 '線断面',
 '倍',
 '倍',
 'システム',
 'データ',
 '実施形態',
 '装置',
 '試験',
 '\u3000ジオキサン',
 '\u3000ジオキサン分解菌',
 '\u3000効果\u3000',
 '\u3000変形例\u3000',
 '\u3000変形例\u3000本願発明',
 '\u3000施工\u3000',
 'ペレット',
 '適用例\u3000',
 '変形例',
 '炭化',
 '発明',
 'あおり補正',
 'あき寸法',
 'あき重ね継手',
 'あたりリットル',
 'あたり',
 'あて板',
 'あと施工',
 'あと施工アンカー',
 'あと施工アンカーボルト',
 'あと施工アンカー工事',
 'あと施工アンカー工法',
 'いし側縁',
 'いも継ぎ',
 'い集水能力',
 'うず電流',
 'おいてつ',
 'おいてつ配置',
 'おかず速やか',
 'おむつ交換',
 'おんマット',
 'かご単位

In [29]:
#単語:文書内単語頻度 の辞書の列つくって、pandasにくっつける
def makeTermsTfDict(terms, vec):
    # dic = {key: val for key, val in zip(terms, vec)  if val } #dfが1より大きいtermの辞書つくる
    dic = {key: val for key, val in zip(terms_list, vec)  }
    dic2 = dict(sorted(dic.items(), key=lambda x:x[1],reverse=True)) #dfの値でソート
    return dic2

In [30]:
xx = makeTermsTfDict(terms_list, c_matrix.getrow(2).toarray()[0])

In [31]:
xx

{'構築工法': 16,
 '作業足場': 6,
 '鋼管設置工程': 5,
 'コンクリート打設工程': 4,
 '円形断面': 4,
 '高所作業': 4,
 'ズレ止め': 2,
 '安全管理': 2,
 '座屈': 2,
 '所定形状': 2,
 '拘束機能': 2,
 '支持構造': 2,
 '構築過程': 2,
 '角形断面': 2,
 '鉄筋加工': 2,
 'リブ付鋼管': 1,
 '一実施例': 1,
 '二発明': 1,
 '作業環境': 1,
 '作業用足場': 1,
 '係止': 1,
 '利用分野': 1,
 '剪断抵抗': 1,
 '垂直上方': 1,
 '垂設': 1,
 '帯鉄筋': 1,
 '拘束効果': 1,
 '揚重機': 1,
 '断面形状': 1,
 '楕円断面': 1,
 '構築予定': 1,
 '構築予定箇所': 1,
 '溶接検査': 1,
 '終了状態': 1,
 '組立設置': 1,
 '解体作業': 1,
 '重ね継ぎ手': 1,
 '重ね継手': 1,
 '重ね継手構造': 1,
 '鉄筋コンクリート構造': 1,
 '閉鎖断面形状': 1,
 '緯度': 0,
 'エルボ': 0,
 'カメラ': 0,
 'ピッチ': 0,
 'フック': 0,
 '付近': 0,
 '位相': 0,
 '傾斜': 0,
 '刻み': 0,
 '反転': 0,
 '回動': 0,
 '回転': 0,
 '変更': 0,
 '対称': 0,
 '屈曲': 0,
 '旋回': 0,
 '移動': 0,
 '程度傾斜': 0,
 '転回': 0,
 '間隔': 0,
 '離間': 0,
 '仰角': 0,
 '湿度': 0,
 '秒': 0,
 'ｃｍ': 0,
 'ｍｍ': 0,
 '××ｃｍ供試体': 0,
 'メチレンビスメチル': 0,
 'コイル出口水温': 0,
 'コイル出口空気温度': 0,
 '冷却水温度': 0,
 '分間遠心分離': 0,
 '圧力': 0,
 '相対湿度': 0,
 'ｒｐｍ': 0,
 '上昇': 0,
 '低下': 0,
 '冷水': 0,
 '加熱': 0,
 '加熱養生': 0,
 '変化': 0,
 '封かん養生': 0,
 '標準養生': 0,
 '水中': 0,
 '水中養生'

In [41]:
#単語:文書内単語頻度 の辞書の列つくって、pandasにくっつける
def makeTermsTfDict(terms, vec):
    # dic = {key: val for key, val in zip(terms, vec)  if val } #dfが1より大きいtermの辞書つくる
    dic = {key: val for key, val in zip(terms, vec)  }
    # dic2 = dict(sorted(dic.items(), key=lambda x:x[1],reverse=True)) #dfの値でソート
    return dic

# df['term'] = pd.Series([makeTermsTfDict(terms_list, vec) for vec in [c_matrix.getrow(i).toarray()[0] for i in range(c_matrix.shape[0])]]) #countされたベクトルの各行を関数に入れていってシリーズに

xx = [makeTermsTfDict(terms_list, vec) for vec in [c_matrix.getrow(i).toarray()[0] for i in range(c_matrix.shape[0])]]

# df['term'] = pd.Series([makeTermsTfDict(terms_list, vec) for vec in [c_matrix.getrow(i).toarray()[0] for i in range(1000)]]) #countされたベクトルの各行を関数に入れていってシリーズに

In [39]:
xx

,patent_name,id,head_inventor,inventor,IPC,headIPC,f_term,FI,createdBy,createdAt,sentence,term
0,Ｚ形仕切付段ボール箱,実願2001-003729,佐藤 浩康,佐藤 浩康,B65D 5/491 (2006.01),B65D 5/491 (2006.01),"3E060AA03,3E060AB05,3E060BC02,3E060CB04,3E060C...","B65D 5/48 101J,B65D 5/498",{佐藤工業株式会社},2001-12-14,【考案の詳細な説明】 【０００１】 【考案の属する技術分野】 本考案は、袋菓子または詰...,"{'連設': 8, '一対': 6, '段ボールシート': 4, '仕切り壁': 3, '箱..."
1,吊りフック,実願2001-002834,石塚 浩次,"石塚 浩次,織田 悦芳,好田 勝之",B66C 1/36 (2006.01),B66C 1/36 (2006.01),"3F004CC01,3F004CD08",B66C 1/36 A,"{社団法人日本海上起重技術協会, 東亜建設工業株式会社, 寄神建設株式会社}",2001-11-16,【考案の詳細な説明】【０００１】【考案の属する技術分野】 本考案は、クレーン等によりコンクリ...,"{'玉掛けワイヤ': 12, '回動': 10, 'メッセンジャーワイヤ': 10, '下げ..."
2,橋脚柱の構築工法,特願2001-120638,大内 一,"大内 一,田中 浩一","E01D 19/02 (2006.01),E01D 21/00 (2006.01)",E01D 19/02 (2006.01),"2D059AA03,2D059CC04","E01D 19/02 ,E01D 21/04 ,E01D 21/00...",{株式会社大林組},2001-11-09,【発明の詳細な説明】【０００１】【産業上の利用分野】この発明は、橋脚柱の構築工法および橋脚柱...,"{'構築工法': 16, '作業足場': 6, '鋼管設置工程': 5, 'コンクリート打設..."
3,コンクリートの運搬装置,特願2001-109004,藤原 宗一,藤原 宗一,"E02B 7/00 (2006.01),B65G 33/12 (2006.01)",E02B 7/00 (2006.01),"3F040BA01,3F040CA04,3F040EA03","E02B 7/00 C,B65G 33/12",{株式会社大林組},2001-12-26,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、コンクリートの運搬装置に...,"{'運搬装置': 19, '螺旋羽根': 11, 'コンクリートプラント': 9, '回転駆..."
4,コンクリートの運搬方法,特願2001-108995,藤原 宗一,藤原 宗一,E02B 7/00 (2006.01),E02B 7/00 (2006.01),NaN,E02B 7/00 C,{株式会社大林組},2001-12-21,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、コンクリートの運搬方法に...,"{'コンクリートプラント': 8, '螺旋羽根': 8, '材料分離': 6, '回転駆動機..."
...,...,...,...,...,...,...,...,...,...,...,...,...
44559,壁高欄の補強方法及び補強構造,特願2019-114410,藤原 保久,"藤原 保久,安藤 直文,清水 宏一朗,近藤 真一,迎 邦博","E01D 19/10 (2006.01),E01D 22/00 (2006.01)",E01D 19/10 (2006.01),"2D059AA21,2D059GG55,2D059GG40,2D059GG02","E01D 19/10 ,E01D 22/00 B","{ファイベックス株式会社, 三井住友建設株式会社}",2021-01-07,【発明の詳細な説明】【技術分野】【０００１】 本発明は、コンクリート床版を有する橋梁、高架橋...,NaN
44560,建物の制振構造及びその構築方法,特願2019-114061,松永 健太郎,"松永 健太郎,新上 浩","E04H 9/02 (2006.01),E04B 1/04 (2006....",E04H 9/02 (2006.01),"2E139AA01,2E139AB03,2E139BD24,2E139BD23,2E139B...","E04H 9/02 321C,E04B 1/04 M,E04H 9/02...",{三井住友建設株式会社},2021-01-07,【発明の詳細な説明】【技術分野】【０００１】 本開示は、建物の制振構造及びその構築方法に関し...,NaN
44561,測位アルゴリズムの設定パラメータ決定方法,特願2019-114049,木村 駿介,木村 駿介,"G01S 19/43 (2010.01),G01S 19/22 (2010....",G01S 19/43 (2010.01),"5J062AA09,5J062CC07,5J062EE04,5J062EE01,5J062DD23","G01S 19/43 ,G01S 19/22 ,G01S 19/28...",{清水建設株式会社},2021-01-07,【発明の詳細な説明】【技術分野】【０００１】 本発明は、測位アルゴリズムにおける最良の設定パ...,NaN
44562,肥料,特願2019-113928,横山 茂輝,"横山 茂輝,袋 昭太,倉澤 響","C05G 3/40 (2020.01),C05B 17/00 (2006.01)",C05G 3/40 (2020.01),"4H061AA01,4H061BB29,4H061EE16,4H061LL26,4H061L...","C05G 3/00 103 ,C05B 17/00 ,C05G 3/40...",{株式会社フジタ},2021-01-07,【発明の詳細な説明】【技術分野】【０００１】 本発明は肥料に関する。特に、本発明はリン及び鉄...,NaN


In [ ]:
# df.to_excel('parcedPatents.xlsx', sheet_name='Sheet1', ascending=False)

In [ ]:
df.to_json('parcedPatents.json', ascending=False)